# Introduction to Python 3, part 2
Hans Petter Langtangen, Joakim Sundnes, Ola Skavhaug, Simon Funke

Date: **Sep 14, 2015**

## Contents

* Classes in Python
  * Class syntax and inheritance
  * Special attributes and methods
  * Static attributes
  * Properties
* Scope - The lifetime of variables
* Testing  

## But first: some general updates

Assignment 3:
* Deadline for assignment 3 is this Sunday (18th Sept.)


Assignment 4:
* Will be published on Sunday
* 3 week deadline (9th Oct) + 1 week peer-review
* Topic: Numerical Python

# Classes in Python

## More information
  * [The Python tutorial, chapter 9](https://docs.python.org/3/tutorial/classes.html)
  * [Think Python, chapters 15](http://greenteapress.com/wp/think-python-2e/)
  * [Effective Computation in Physics (Python 2)](http://shop.oreilly.com/product/0636920033424.do)
  * Ch. 2 in *Illustrating Python via bioinformatics examples*, [PDF](http://www.uio.no/studier/emner/matnat/ifi/INF3331/h14/lectures/bioinf-py-2.pdf) (Python 2)

## Classes in Python

* Similar class concept as in Java and C++
* Class programming is easier and faster than in C++ and Java:
  * All functions are virtual
  * No private/protected variables (the effect can be "simulated")
* Single and multiple inheritance
* Everything in Python is an object, even the source code

In [ ]:
a = 1
help(a)

## Declaring a new Python class

Declare a new class `MyBase`:

In [ ]:
class MyBase(object):
    """ Docstring """
    def __init__(self,i, j):  # constructor
        self.i = i            # class variable
        self.j = j            # class variable

    def write(self):          # member function
        print('MyBase: i={0}, j={1}'.format(self.i, self.j))   # access 
                                                    # member attributes

* `MyBase` is derived from the `object` base class.
* `self` is a reference to this object.
* Class variables are prefixed by `self`:
  `self.i`, `self.j`
* Class functions take `self` as first argument.

## Using a Python class

Instantiating the class calls the `__init__` constructor and returns a new object:

In [ ]:
my_object = MyBase(6, 9)

Accessing class attributes:

In [ ]:
my_object.i
my_object.write()

**Note**: When calling class functions, the `self` argument in the declaration is neglected:

Use `dir` to list all attributes of an object:

In [ ]:
dir(my_object)

## Implementing a subclass
Class `MySub` is a subclass of `MyBase`:

In [2]:
class MySub(MyBase):     # Inherit MySub for MyBase

    def __init__(self, i, j, k):     # Constructor
        MyBase.__init__(self, i, j)  # Call the base constructur
        self.k = k                   # Add a new class variable

    def write(self):
        print('MyBase: i={0}, j={1}, k={2}'.format(self.i, self.j, \
               self.k))

NameError: name 'MyBase' is not defined

Example usage:

In [ ]:
# make a MySub instance
mysub_obj = MySub(7, 8, 9)
mysub_obj.write()   # will call MySub's write

## Comment on object-orientation
Consider

In [ ]:
# this function works with any object that has a write func
def write(v):
    v.write()

write(mysub_obj)   # mysub_obj is MySub instance

* In C++/Java we would declare `v` as a `MyBase` reference and rely on `mysub_obj.write()` to call the virtual function `write` in `MySub`.
* The same works in Python, but we do not need inheritance and virtual functions here: `v.write()` will work for *any* object `v` that has a callable attribute `write` that takes no arguments

## Testing on the class type
Test if an instance is of class type:
```python
if isinstance(i2, MySub):
    # treat i2 as a MySub instance
```

Test if a class is a subclass of another:
```python
if issubclass(MySub, MyBase):
    # MySub inherits from MyBase 
```

Test if two objects are of the same class:
```python
if inst1.__class__ is inst2.__class__:
```
(`myobj.__class__` refers the class object of instance `myobj`)

## Private/non-public data
* In prinicpal, there is no technical way of preventing users from manipulating data and methods in an object:

In [ ]:
my_object.write()
my_object.write = lambda: print("I dont care")
my_object.write()

## Private/non-public data

But there are some conventions: 
* Attributes **starting with one underscore** are treated as non-public ("protected").
* Names **starting with one double underscore** are considered strictly private (Python mangles class name with method name in this case: `obj.__some` has actually the name `_classname__some`).
* Names **starting and ending with double underscores** are special methods and attributes (discussed later)

Here is an example...

## Private/non-public data

In [ ]:
class MyClass(object):
    def __init__(self):
        self._a = False    # non-public
        self.b = 0         # public
        self.__c = 0       # private
        
    def __hidden(self): # a private function
        pass

In [ ]:
m = MyClass()
m._a
m.b

In [3]:
m._MyClass__c         # m.__c does not exist
m._MyClass__hidden()  # m.__hidden does not exist

NameError: name 'm' is not defined

In [ ]:
dir(m)

## Special attributes

## Special attributes

Let's consider again our two basic classes:

In [4]:
i1 = MyBase(1, 2)
i2 = MySub(1, 2, 3)

NameError: name 'MyBase' is not defined

Listing all methods and attributes with `dir` reveils that these instances have already some *special* attributes:

In [ ]:
dir(i2)

## Let's inspect some of these special attributes

In [ ]:
i1.__dict__

In [ ]:
i2.__dict__

**Conclusion**: `__dict__` returns a dictionary of user-defined attributes.

Name of class, name of method:

In [ ]:
i2.__class__          # class of object

In [ ]:
i2.__class__.__name__ # name of class

In [ ]:
i2.write.__name__     # name of method

## Special methods
* Special methods have leading and trailing double underscores.
* If implemented, Python calls these special  methods on certain operations. 
* Here are some operations defined by special methods:
```python
str(a)            # calls a.__str__(), called also with print(a)!
len(a)            # calls a.__len__()
c = a*b           # calls c = a.__mul__(b)
a = a+b           # calls a = a.__add__(b)
a += c            # calls a.__iadd__(c)
d = a[3]          # calls d = a.__getitem__(3)
a[3] = 0          # calls a.__setitem__(3, 0)
f = a(1.2, True)  # calls f = a.__call__(1.2, True)
if a:             # calls if a.__len__()> 0: or if a.__nonzero__():
a == b            # calls a.__eq__(self, b)
```

A complete list is avaible here: [Python reference - Data model](https://docs.python.org/3/reference/datamodel.html)

## Example: functions with extra parameters

Suppose we need a function of `x` and `y` with three additional parameters `a`, `b`, and `c`:

In [ ]:
def f(x, y, a, b, c):
    return a + b*x + c*y*y

Suppose we need to send this function to another function

In [ ]:
def gridvalues(f, xcoor, ycoor, file):
    for i in range(len(xcoor)):
        for j in range(len(ycoor)):
            fval = f(xcoor[i], ycoor[j])
            file.write('%g %g %g\n' % (xcoor[i], ycoor[j], fval))

* `func` is expected to be a function of `x` and `y` only (many libraries need to make such assumptions!)
* How can we send our `f` function to `gridvalues`?

## Possible (inferior) solution
**Bad solution 1**: global parameters

```python
global a, b, c
#...
def f(x, y):
    return a + b*x + c*y*y

#...
a = 0.5;  b = 1;  c = 0.01
gridvalues(f, xcoor, ycoor, somefile)
```

Global variables are usually considered evil!

## Possible (inferior) solution
**Bad solution 2**: keyword arguments with default values

```python
def f(x, y, a=0.5, b=1, c=0.01):
    return a + b*x + c*y*y

# ...
gridvalues(f, xcoor, ycoor, somefile)
```
useless for other values of `a`, `b`, `c`

## Solution: class with `__call__` operator
  1. Make a class with function behavior instead of a pure function.
  2. Make the parameters class attributes.
  3. Implement the special `__call__`  function.
  
Now, instances can be called as ordinary functions, but with *x* and *y* as the only formal arguments

In [ ]:
class F(object):
    def __init__(self, a=1, b=1, c=1):
        self.a = a
        self.b = b
        self.c = c

    def __call__(self, x, y):    # special method!
        return self.a + self.b*x + self.c*y*y

In [ ]:
f = F(a=0.5, c=0.01)
# can now call f as
v = f(0.1, 2)
# ...
gridvalues(f, xcoor=[], ycoor=[], file="somefile")

## Class variables
Static data (or class variables) are common to all class instances.

In [ ]:
class Point(object):
    " A class representing a 2D point "

    counter = 0 # static variable, counts no of instances
    
    def __init__(self, x, y):
        self.x = x
        self.y = y;
        Point.counter += 1

In [ ]:
for i in range(1000):
        p = Point(i*0.01, i*0.001)

Access without instance:

In [ ]:
Point.counter     

Access with instance:

In [ ]:
p.counter      

## Static methods
Python classes also allow static methods (that is, methods that can be called without having an instance):

In [ ]:
class Point(object):
    " A class representing a 2D point "
    _counter = 0
    
    def __init__(self, x, y):
        self.x = x; self.y = y
        Point._counter += 1
        
    def ncopies():  # No need for a self argument for static methods
        return Point._counter

    ncopies = staticmethod(ncopies)

`ncopies` can be called directly from the class:

In [ ]:
Point.ncopies()

... or from an instance

In [ ]:
p = Point(0, 0)
p.ncopies()

In [ ]:
Point.ncopies()

# Attributes access and properties

## Attribute access

Attributes can be directly accessed and changed from an instance:

In [ ]:
p = Point(0, 0)
p.x = 2       
a = p.x

Alternative approach via get/set functions:

In [ ]:
class A(object):
    def set_attr1(self, attr1):
        self._attr1 = attr1        # underscore => non-public variable
        #self._update(self._attr1)  # update internal data too

    def get_attr1(self):
        # do some internal stuff
        return self._attr1

my_object = A()
my_object.set_attr1(True)    # Not very Pythonic :(
a = my_object.get_attr1()    # Not very Pythonic :(

Advantage: get/set functions allow finer access control (e.g. type-checking, read-only variables). 

**But**: Tedious to write! Use properties instead!

## Properties
Python 2.3 introduced "intelligent" assignment operators, known as
*properties*. 

With properties, assignments may imply a function call:

```python
x.set_data(mydata)
yourdata = x.get_data()
```
can be made equivalent to
```python
x.data = mydata
yourdata = x.data
```

## Properties 

Example construction:

In [ ]:
class MyClass(object):  
    def set_data(self, d):
        print("Setting data")
        self._data = d
        # update other data structures if necessary...

    def get_data(self):
        print("Reading data")
        # ...
        return self._data

    data = property(fget=get_data, fset=set_data)

In [ ]:
inst = MyClass()
inst.data = True
x = inst.data

## Attribute access - recommended style

  * Use direct access if user is allowed to read *and* assign values to the attribute

  * Use properties to restrict access, with a corresponding underlying non-public class attribute

  * Use properties when assignment or reading requires a set of associated operations

  * Never use get/set functions explicitly

  * Attributes and functions are somewhat interchanged in this scheme $\Rightarrow$ that's why we use the same naming convention

# Scope

## Scope
The scope defines how long variables in Python *live*.

#### Example
Function arguments and variables declared inside the function have *local scope*. Once the function finishes, these variables are freed.

#### Example
Variables defined outside the function have *global scope*. These variables can be accessed and changed by the function and are accessible after the function returns.

## Code example for scopes

In [ ]:
def f():
    a = 1  # local scope
    print(a)
 
f()

## Code example for scopes

In [ ]:
# global scope
a = 1 

def f():
    print(a)

f()

## Code example for scopes

In [ ]:
# global scope
a = 1 

def f():
    print(a)

a = "s"
f()

## Code example for scopes

In [ ]:
# global scope
a = ["Hello"]

def f():
    def g():
        a.append("world?")
    g()

f()
print(a)

## Code example for scopes

In [ ]:
# global scope
a = 1 

def f(x):
    a = 2             # local variable

class B(object):
    b = 3             # static class attribute

    def __init__(self):
        self.a = 3    # class attribute

    def scopes(self):
        a = 4         # local (method) variable

## Namespaces for exec and eval
`exec` and `eval` may take dictionaries for the global and local namespace:

In [ ]:
code = "a=1"
exec(code, globals(), locals())
expr = "a+1"
eval(expr, globals(), locals())

Example:

In [ ]:
a = 8;  b = 9
d = {'a':1, 'b':2}
eval('a + b', d)  # yields 3

and

from math import *
d['b'] = pi
eval('a+sin(b)', globals(), d)  # yields 1

Creating such dictionaries can be handy

# Testing in Python
Jonathan Feinberg

## Why should we test?

* To check correctness of software.
* To ensure that future changes do not break functionality.
* To check if the software runs succesfully in a different environment (newer Python version, upgraded libraries, different operating system)

## A few options in Python

* [Unittest](https://docs.python.org/3/library/unittest.html)

* [Doctest](https://docs.python.org/3/library/doctest.html)
* [Py.test](http://pytest.org/) (will be used here)

## How to use py.test

Say you have a function `func` in a file that needs testing:

In [ ]:
# script.py 
def func(x):
    if x < 0:
        return -x
    else:
        return x

Create a associated test file `test_script.py`:

In [ ]:
# test_script.py
from script import func  # Import the function 

def test_funcs():        # py.test will automatically run all functions starting with test_
    assert func(-3) == 3 # Add some tests here...
    assert func(5) == 5  # If one of the assert's are false,
    assert func(0) == 0  # the test will fail

In [ ]:
!py.test test_script.py -v

# Using py.test
Now, lets make a mistake in our func implementation ...

In [ ]:
!sed -i "s/-x/x/g" script.py
!cat script.py

and run the tests again

In [ ]:
!py.test test_script.py -v

## Good practices
* Add new test while you develop new features.
* Make each test an unique stand alone example.
* Making tests resource undemanding.
* Run test suite before each commit-push.
* Make test function names descriptive.
* Quick way to learn other peoples code is through test suits.